# Mutagenesis Example in Python

Following example is the same as the julia version of the 
Following example demonstrates learning to predict the mutagenicity on Salmonella typhimurium using the [JuliaPy/pyjulia](https://github.com/JuliaPy/pyjulia), and demonstrates how can the JsonGrinder.jl be used entirely from python.

We start by installing the pyjulia and use it to install the JsonGrinder and few other packages we need for the example.

In [2]:
!pip install julia

In [3]:
import sys
sys.version

'3.9.10 | packaged by conda-forge | (main, Feb  1 2022, 21:24:11) \n[GCC 9.4.0]'

In [4]:
import julia

In [5]:
julia.install()

[ Info: Julia version info


Julia Version 1.7.2
Commit bf53498635 (2022-02-06 15:21 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  uname: Linux 5.10.16.3-microsoft-standard-WSL2 #1 SMP Fri Apr 2 22:23:49 UTC 2021 x86_64 x86_64
  CPU: Intel(R) Core(TM) i7-9850H CPU @ 2.60GHz: 
              speed         user         nice          sys         idle          irq
       #1  2592 MHz     195979 s       1237 s     173734 s    7772796 s          0 s
       #2  2592 MHz     258408 s       1058 s     168196 s    7744576 s          0 s
       
  Memory: 1.9366111755371094 GB (62.88671875 MB free)
  Uptime: 871679.82 sec
  Load Avg:  33.92  57.72  34.75
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-12.0.1 (ORCJIT, skylake)
Environment:
  JULIA_DEPOT_PATH = /opt/julia
  JULIA_PKGDIR = /opt/julia
  JULIA_VERSION = 1.7.2
  PATH = /opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin
  TERM = xterm-color
  HOME = /home/jovyan
  XDG_CACHE_HOME = /home/jovyan/.cache/
  JULIA_DEPOT_PATH = /opt/

[ Info: Julia executable: /opt/julia-1.7.2/bin/julia
[ Info: Trying to import PyCall...
┌ Info: PyCall is already installed and compatible with Python executable.
│ 
│ PyCall:
│     python: /opt/conda/bin/python
│     libpython: /opt/conda/lib/libpython3.9.so.1.0
│ Python:
│     python: /opt/conda/bin/python
└     libpython: 


In [6]:
from julia import Julia
Julia(compiled_modules=False)
#Julia(runtime='C:\\Users\\racinsky\\AppData\\Local\\Programs\\Julia-1.7.2\\bin\\julia.exe')

/opt/conda/lib/python3.9/site-packages/julia/core.py:703: FutureWarning: Accessing `Julia().<name>` to obtain Julia objects is deprecated.  Use `from julia import Main; Main.<name>` or `jl = Julia(); jl.eval('<name>')`.
  warnings.warn(


In [7]:
from julia import Base
Base.VERSION

<PyCall.jlwrap 1.7.2>

We install the packages using the julia package manager called from python

Executing the following line may take few minutes, because the package manager needs to resolve proper versions.

In [ ]:
from julia import Pkg
Pkg.add("MLDatasets")
Pkg.add("Flux")
Pkg.add("Mill")
Pkg.add("MLDataPattern")
Pkg.add("Statistics")
Pkg.add(name="JsonGrinder", rev="master")


Here we include libraries all necessary libraries

Executing the following cell may take few minutes to finish, which is expected, because there is lots of code being precompiled.

In [ ]:
from julia import JsonGrinder, MLDatasets, Flux, Mill, MLDataPattern, Statistics

Here we load all samples.

In order to download datasets automatically from jupyter notebook, make sure the environment variable `DATADEPS_ALWAYS_ACCEPT` is present and set to true. E.g. using this if you run the notebook from docker compose
```
services:
  ...
    environment:
      - DATADEPS_ALWAYS_ACCEPT=true
```


In [10]:
train_x, train_y = MLDatasets.Mutagenesis.traindata();
test_x, test_y = MLDatasets.Mutagenesis.testdata();

We define some basic parameters for the construction and training of the neural network. Minibatch size is self-explanatory, iterations is number of iterations of gradient descent Neurons is number of neurons in hidden layers for each version of part of the neural network.

In [11]:
minibatchsize = 100
iterations = 5_000
neurons = 20

We create the schema of the training data, which is the first important step in using the JsonGrinder. This computes both the structure (also known as JSON schema) and histogram of occurrences of individual values in the training data.

In [12]:
sch = JsonGrinder.schema(train_x)
sch

<PyCall.jlwrap DictEntry>

Then we use it to create the extractor converting jsons to Mill structures. The suggestextractor is executed below with default setting, but it allows you heavy customization. We also prepare list of classes. This classification problem is two-class, but we want to infer it from labels.

In [ ]:
extractor = JsonGrinder.suggestextractor(sch)
labelnames = Base.unique(train_y)

# Create the model
We create the model reflecting structure of the data

In [ ]:
model = Mill.reflectinmodel(sch, extractor,
    lambda layer: Flux.Dense(layer, neurons, Flux.relu),
    lambda bag: Mill.SegmentedMeanMax(bag),
    fsm = {"": lambda layer: Flux.Dense(layer, Base.length(labelnames))},
)
model

We convert jsons to mill data samples and prepare list of classes. This classification problem is two-class, but we want to infer it from labels. The extractor is callable, so we can pass it vector of samples to obtain vector of structures with extracted features.

In [ ]:
train_data = list(map(extractor, train_x))
test_data = list(map(extractor, test_x))
test_data

# Train the model
Then, we define few handy functions and a loss function, which is categorical crossentropy in our case.

In [64]:
def inference(x): return model(x).data
def loss(x,y): return Flux.logitcrossentropy(inference(x), Flux.onehotbatch(y, labelnames))
def accuracy(x,y): return mean(labelnames[Flux.onecold(inference(x))] == y)

And we can add a callback which will be printing train and test accuracy during the training and then we can start trining

In [65]:
def cb():
    train_acc = accuracy(train_data, train_y)
    test_acc = accuracy(test_data, test_y)
    println("accuracy: train = $train_acc, test = $test_acc")

Lastly we turn our training data to minibatches, and we can start training

In [66]:
minibatches = MLDataPattern.RandomBatches((train_data, train_y), size = minibatchsize, count = iterations)

In [67]:
ps = Flux.params(model)
x, y = Base.first(minibatches)
x, y

([<PyCall.jlwrap ProductNode>,
  <PyCall.jlwrap ProductNode>,
  <PyCall.jlwrap ProductNode>,
  <PyCall.jlwrap ProductNode>,
  <PyCall.jlwrap ProductNode>,
  <PyCall.jlwrap ProductNode>,
  <PyCall.jlwrap ProductNode>,
  <PyCall.jlwrap ProductNode>,
  <PyCall.jlwrap ProductNode>,
  <PyCall.jlwrap ProductNode>,
  <PyCall.jlwrap ProductNode>,
  <PyCall.jlwrap ProductNode>,
  <PyCall.jlwrap ProductNode>,
  <PyCall.jlwrap ProductNode>,
  <PyCall.jlwrap ProductNode>,
  <PyCall.jlwrap ProductNode>,
  <PyCall.jlwrap ProductNode>,
  <PyCall.jlwrap ProductNode>,
  <PyCall.jlwrap ProductNode>,
  <PyCall.jlwrap ProductNode>,
  <PyCall.jlwrap ProductNode>,
  <PyCall.jlwrap ProductNode>,
  <PyCall.jlwrap ProductNode>,
  <PyCall.jlwrap ProductNode>,
  <PyCall.jlwrap ProductNode>,
  <PyCall.jlwrap ProductNode>,
  <PyCall.jlwrap ProductNode>,
  <PyCall.jlwrap ProductNode>,
  <PyCall.jlwrap ProductNode>,
  <PyCall.jlwrap ProductNode>,
  <PyCall.jlwrap ProductNode>,
  <PyCall.jlwrap ProductNode>,
  <PyCal

In [73]:
loss(x, y)

0.7206074595451355

In [74]:
gs = Flux.gradient(lambda: loss(x, y), ps)

RuntimeError: <PyCall.jlwrap (in a Julia function called from Python)
JULIA: Compiling Tuple{typeof(PyCall._pycall!), PyObject, PyObject, Tuple{}, Int64, Ptr{Nothing}}: try/catch is not supported.
Stacktrace:
  [1] error(s::String)
    @ Base ./error.jl:33
  [2] instrument(ir::IRTools.Inner.IR)
    @ Zygote /opt/julia/packages/Zygote/cCyLF/src/compiler/reverse.jl:121
  [3] #Primal#20
    @ /opt/julia/packages/Zygote/cCyLF/src/compiler/reverse.jl:202 [inlined]
  [4] Zygote.Adjoint(ir::IRTools.Inner.IR; varargs::Nothing, normalise::Bool)
    @ Zygote /opt/julia/packages/Zygote/cCyLF/src/compiler/reverse.jl:315
  [5] _generate_pullback_via_decomposition(T::Type)
    @ Zygote /opt/julia/packages/Zygote/cCyLF/src/compiler/emit.jl:101
  [6] #s5902#1220
    @ /opt/julia/packages/Zygote/cCyLF/src/compiler/interface2.jl:28 [inlined]
  [7] var"#s5902#1220"(::Any, ctx::Any, f::Any, args::Any)
    @ Zygote ./none:0
  [8] (::Core.GeneratedFunctionStub)(::Any, ::Vararg{Any})
    @ Core ./boot.jl:580

In [ ]:
gs = Flux.gradient(lambda: Flux.logitcrossentropy(inference(x), Flux.onehotbatch(y, labelnames)), ps)

In [43]:
Flux.Optimise.train_b(lambda xy: loss(xy[0], xy[1]), Flux.params(model), minibatches, Flux.ADAM(), cb = Flux.throttle(cb, 2))

RuntimeError: <PyCall.jlwrap (in a Julia function called from Python)
JULIA: Compiling Tuple{typeof(PyCall._pycall!), PyObject, PyObject, Tuple{SubArray{Mill.ProductNode{NamedTuple{(:lumo, :inda, :logp, :ind1, :atoms), Tuple{Mill.ArrayNode{Flux.OneHotArray{UInt32, 0x00000063, 1, 2, Vector{UInt32}}, Nothing}, Mill.ArrayNode{Flux.OneHotArray{UInt32, 0x00000002, 1, 2, Vector{UInt32}}, Nothing}, Mill.ArrayNode{Flux.OneHotArray{UInt32, 0x0000003f, 1, 2, Vector{UInt32}}, Nothing}, Mill.ArrayNode{Flux.OneHotArray{UInt32, 0x00000003, 1, 2, Vector{UInt32}}, Nothing}, Mill.BagNode{Mill.ProductNode{NamedTuple{(:element, :bonds, :charge, :atom_type), Tuple{Mill.ArrayNode{Flux.OneHotArray{UInt32, 0x00000007, 1, 2, Vector{UInt32}}, Nothing}, Mill.BagNode{Mill.ProductNode{NamedTuple{(:element, :bond_type, :charge, :atom_type), Tuple{Mill.ArrayNode{Flux.OneHotArray{UInt32, 0x00000007, 1, 2, Vector{UInt32}}, Nothing}, Mill.ArrayNode{Flux.OneHotArray{UInt32, 0x00000004, 1, 2, Vector{UInt32}}, Nothing}, 

In [75]:
def actual(x): return 4*x + 2

In [88]:
x_train, x_test = [list(range(0,5))], [list(range(6,10))]
x_train, x_test

([[0, 1, 2, 3, 4]], [[6, 7, 8, 9]])

In [90]:
y_train, y_test = list(map(actual, x_train[0])), list(map(actual, x_test[0]))
y_train, y_test

([2, 6, 10, 14, 18], [26, 30, 34, 38])

In [91]:
model = Flux.Dense(1, 1)

In [92]:
model(x_train)

array([[0.       , 0.9608712, 1.9217424, 2.8826137, 3.8434849]],
      dtype=float32)

In [94]:
def loss(x, y): return Flux.Losses.mse(model(x), y)

In [95]:
loss(x_train, y_train)

99.10480499267578

In [97]:
parameters = Flux.params(model)

In [99]:
data = [(x_train, y_train)]
Flux.train_b(loss, parameters, data, Flux.ADAM())

RuntimeError: <PyCall.jlwrap (in a Julia function called from Python)
JULIA: Compiling Tuple{typeof(PyCall._pycall!), PyObject, PyObject, Tuple{Matrix{Int64}, Vector{Int64}}, Int64, Ptr{Nothing}}: try/catch is not supported.
Stacktrace:
  [1] error(s::String)
    @ Base ./error.jl:33
  [2] instrument(ir::IRTools.Inner.IR)
    @ Zygote /opt/julia/packages/Zygote/cCyLF/src/compiler/reverse.jl:121
  [3] #Primal#20
    @ /opt/julia/packages/Zygote/cCyLF/src/compiler/reverse.jl:202 [inlined]
  [4] Zygote.Adjoint(ir::IRTools.Inner.IR; varargs::Nothing, normalise::Bool)
    @ Zygote /opt/julia/packages/Zygote/cCyLF/src/compiler/reverse.jl:315
  [5] _generate_pullback_via_decomposition(T::Type)
    @ Zygote /opt/julia/packages/Zygote/cCyLF/src/compiler/emit.jl:101
  [6] #s5902#1220
    @ /opt/julia/packages/Zygote/cCyLF/src/compiler/interface2.jl:28 [inlined]
  [7] var"#s5902#1220"(::Any, ctx::Any, f::Any, args::Any)
    @ Zygote ./none:0
  [8] (::Core.GeneratedFunctionStub)(::Any, ::Vararg{An